# 2値ラベル系の統一フレームワーク：$\Omega$ と $E$ の置換

このノートブックは、以下の分野を一貫して扱うための「統一フレームワーク」を定義します。

1.  **検査・判定・分類**（ベイズ推定、感度・特異度）
2.  **抽出・サンプリング**（有限母集団、非復元抽出、超幾何分布）
3.  **検定**（比率の差、適合度）
4.  **確率過程**（逐次観測、ランダムウォーク）

これらは一見異なる問題に見えますが、すべて**「結果空間 $\Omega$ 上のイベント $E$ をどう定義し、どう計算するか」**という共通の構造で解くことができます。特に、対象が「0か1か（All-or-Nothing）」である場合、この構造は極めて強力です。


## 1. コア・アブストラクション：$\Omega, E, I$

すべての出発点は以下の3つです。

### (1) 結果空間 $\Omega$（登場人物の整列）
起こりうる「世界のあり方」や「観測結果」のすべてのパターンを並べた集合。
例：
- 検査：$\Omega = \{(d, r_1, r_2) \mid d\in\{0,1\}, r_i\in\{+,-\}\}$
- 抽出：$\Omega = \{(x_1, x_2, \dots, x_n) \mid x_i \in \{0,1\}\}$

### (2) イベント $E$（条件・観測）
$\Omega$ の部分集合。問題文で与えられた「事実」や「条件」。
**「解く」とは、この $E$ を問題に合わせて差し替える作業に他なりません。**

### (3) 2値ラベル（指示変数） $I$ または $D$
注目している属性が「ある(1)」か「ない(0)」かを表す変数。
- ベイズでは：真の状態 $D \in \{0, 1\}$（病気か否か）
- サンプリングでは：抽出された個体の値 $X_i \in \{0, 1\}$（当たりかハズレか）


## 2. 応用1：ベイズ推定（観測 $E$ による確率の更新）

ベイズの定理は、$\Omega$ 全体を「$D=1$ の世界」と「$D=0$ の世界」に分割し、観測されたイベント $E$ がどちらの世界で起きやすいかを比較することです。

### 構造
$$P(D=1 \mid E) = \frac{P(E \mid D=1)P(D=1)}{P(E \mid D=1)P(D=1) + P(E \mid D=0)P(D=0)}$$

ここで重要なのは、**イベント $E$ を $\cap$（かつ）で積み上げていく**感覚です。

### 逐次更新（漸化式）
観測が $R_1, R_2, \dots$ と続く場合、イベントは
$$E_t = E_{t-1} \cap \{R_t = r_t\}$$
と更新されます。これを尤度（Likelihood） $L_t(d) = P(E_t \mid D=d)$ の更新式として定式化すると、反射的に計算できます。

$$L_t(d) = P(R_t \mid R_{1:t-1}, D=d) \times L_{t-1}(d)$$

これにより、**「$E$ の差し替え」だけであらゆるベイズ問題（検査、フィルタリングなど）が同型になります。**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import japanize_matplotlib
import ipywidgets as widgets
from IPython.display import display

# ベイズ更新の可視化：Eを積み上げると確率はどう収束するか
def bayes_update_demo(true_positive=0.9, false_positive=0.1, n_steps=10):
    # 真の状態 D=1 とする
    # 観測データ生成：確率 true_positive で 1(陽)、そうでなければ 0(陰)
    np.random.seed(42)
    observations = np.random.choice([1, 0], size=n_steps, p=[true_positive, 1-true_positive])
    
    # 事前確率
    p_d1 = 0.5
    p_d0 = 0.5
    
    # 尤度（初期値）
    L1 = 1.0
    L0 = 1.0
    
    posteriors = [0.5]
    
    for r in observations:
        # Eの差し替え（尤度の更新）
        if r == 1:
            L1 *= true_positive
            L0 *= false_positive
        else:
            L1 *= (1 - true_positive)
            L0 *= (1 - false_positive)
            
        # 事後確率計算（正規化）
        prob = (L1 * p_d1) / (L1 * p_d1 + L0 * p_d0)
        posteriors.append(prob)
        
    # プロット
    plt.figure(figsize=(10, 4))
    plt.plot(posteriors, marker='o', label='P(D=1|E)')
    plt.hlines(1.0, 0, n_steps, linestyles='dashed', colors='gray', label='真の値')
    plt.title(f'イベントEの積み上げによる事後確率の収束 (TPR={true_positive}, FPR={false_positive})')
    plt.xlabel('観測回数')
    plt.ylabel('P(D=1)')
    plt.ylim(-0.1, 1.1)
    plt.grid(True)
    plt.legend()
    plt.show()

# 主要パラメータを操作できるインタラクティブ版
w_true_positive = widgets.FloatSlider(value=0.9, min=0.5, max=1.0, step=0.01, description='TPR')
w_false_positive = widgets.FloatSlider(value=0.1, min=0.0, max=0.5, step=0.01, description='FPR')
w_n_steps = widgets.IntSlider(value=10, min=5, max=50, step=1, description='観測回数')

out = widgets.interactive_output(
    bayes_update_demo,
    {
        'true_positive': w_true_positive,
        'false_positive': w_false_positive,
        'n_steps': w_n_steps,
    },
)
display(widgets.HBox([w_true_positive, w_false_positive, w_n_steps]))
display(out)


## 3. 応用2：有限母集団サンプリング（All-or-Nothingの抽出）

「非復元抽出」も、$\Omega$ と $E$ で記述すると、**共分散（Covariance）** の問題として整理できます。

- 母集団：大きさ $N$、その中に「当たり(1)」が $M$ 個。
- 抽出：$n$ 回引く。$i$ 回目の結果を $X_i \in \{0, 1\}$ とする。
- 関心事：合計 $S = \sum X_i$ の分散 $Var(S)$。

### 独立性との違い（$E$ の交わり）
復元抽出（独立）なら $P(X_i=1 \cap X_j=1) = P(X_i=1)P(X_j=1)$ です。
しかし非復元（有限母集団）では、**「1回目で当たりを引くと、2回目の当たりが減る」**ため、同時確率は積になりません。

$$P(X_i=1 \cap X_j=1) = \frac{M}{N} \times \frac{M-1}{N-1} \neq \left(\frac{M}{N}\right)^2$$

この「ズレ」が共分散 $Cov(X_i, X_j)$ を生み、それが分散公式の「有限母集団補正項」として現れます。

$$Var\left(\sum X_i\right) = \sum Var(X_i) + \sum_{i \neq j} Cov(X_i, X_j)$$

つまり、**「All-or-Nothing（0/1）」の非独立性を、イベントの共通部分（$\cap$）の確率として処理する**ことで、超幾何分布などの複雑な公式を暗記せずとも導出が可能になります。

## 4. 応用3：検定（標準化への接続）

比率の検定も、この $I$（指示変数）の和の分布の問題です。

$$Z = \frac{\hat{p} - p_0}{\sqrt{Var(\hat{p})}} = \frac{\sum I_i - np_0}{\sqrt{Var(\sum I_i)}}$$

ここで $Var(\sum I_i)$ を計算する際に、独立（二項分布）なのか、非復元（超幾何分布）なのかによって、分母に補正が入るかどうかが決まります。これも「$E$ の構造（独立か従属か）」の違いに過ぎません。

## 5. 結論：すべては「代入」である

このチャプター（フレームワーク）の要点は以下の通りです。

1.  まず **$\Omega$（登場人物）** を定義する。
2.  次に **$E$（注目するイベント）** を定義する。
    - 陽性が出た？ ($R=+$)
    - $k$ 個当たりが出た？ ($\sum X = k$)
3.  **計算ルール（独立か、非復元か）** に従って、確率 $P(E)$ や条件付き確率 $P(D|E)$ を計算する。

この手順を固定することで、問題設定が変わっても（ベイズでもサンプリングでも）、同じ思考プロセスで「最短」で解にたどり着くことができます。

## 6. 実践テンプレート：医療検査（感度・偽陽性率）

最も頻出する「2回検査」「繰り返し観測」の問題を、このフレームワークで標準化したテンプレートです。
記号を固定することで、計算ミス（添字のズレや条件の付け忘れ）を完全に防ぎます。

### (1) 記号の固定
- **真の状態**：$D \in \{1, 0\}$ ($1=\text{感/疾患あり}, 0=\text{非/健常}$)
- **事前確率**：$p = P(D=1)$,
- **観測（検査結果）**：$R_i \in \{1, 0\}$ ($1=\text{陽性}, 0=\text{陰性}$)

### (2) パラメータ（条件付き確率）
以下のように「$s$ (sensitivity/true positive)」と「$f$ (false positive)」で対にして定義します。

**1回目（レイヤー1）**
$$s_1 = P(R_1=1 \mid D=1) \quad (\text{感度})$$
$$f_1 = P(R_1=1 \mid D=0) \quad (\text{偽陽性率})$$

**2回目（レイヤー2：1回目陽性の世界での条件付きなど）**
$$s_{2|1+} = P(R_2=1 \mid R_1=1, D=1)$$
$$f_{2|1+} = P(R_2=1 \mid R_1=1, D=0)$$

※ 独立な検査なら $s_{2|1+} = s_2$ ですが、従属（連鎖）する場合もこの記法なら対応できます。

### (3) 最短解答フロー

**Step 1: イベント $E$ を定義する**
例：2回連続で陽性が出た場合
$$E = \{R_1=1\} \cap \{R_2=1\}$$

**Step 2: 尤度 $L(d) = P(E \mid D=d)$ を積で作る**
条件を「新しい順（左）→古い順（右）」に掛けていく感覚で反射的に作ります。

- 疾患あり($D=1$)の世界：
  $$L(1) = P(R_2=1 \mid R_1=1, D=1) \times P(R_1=1 \mid D=1) = s_{2|1+} \times s_1$$
- 疾患なし($D=0$)の世界：
  $$L(0) = P(R_2=1 \mid R_1=1, D=0) \times P(R_1=1 \mid D=0) = f_{2|1+} \times f_1$$

**Step 3: 事後確率 $q = P(D=1 \mid E)$ を計算する**
$$q = \frac{L(1)p}{L(1)p + L(0)(1-p)} = \frac{1}{1 + m}, \quad m = \frac{L(0)(1-p)}{L(1)p}$$

### 補足：計算のコツ
実戦（CBT/PBT）では、分母分子に共通する $10^{-k}$ などを早めに約分し、比 $m$（オッズ比の逆数的なもの）を計算するのが最速です。

#### 実装例
以下は、このテンプレートを用いたPython関数です。

In [ ]:
def solve_medical_test(p_disease, sensitivity_1, false_pos_1, sensitivity_2_cond=None, false_pos_2_cond=None):
    """
    2回検査（または1回検査）のベイズ更新を計算するテンプレート関数
    
    Parameters:
    - p_disease (p): 事前確率 P(D=1)
    - sensitivity_1 (s1): 1回目の感度 P(R1=1|D=1)
    - false_pos_1 (f1): 1回目の偽陽性率 P(R1=1|D=0)
    - sensitivity_2_cond (s2|1+): 2回目の条件付き感度 (Noneなら1回のみ)
    - false_pos_2_cond (f2|1+): 2回目の条件付き偽陽性率
    """
    
    # 1. 尤度 L(d) の計算（積の形）
    # D=1の世界
    L1 = sensitivity_1
    if sensitivity_2_cond is not None:
        L1 *= sensitivity_2_cond
        
    # D=0の世界
    L0 = false_pos_1
    if false_pos_2_cond is not None:
        L0 *= false_pos_2_cond
        
    # 2. 事後確率の計算
    # 分子 (True Positive側)
    numerator = L1 * p_disease
    # 分母 (Total Positive)
    denominator = numerator + L0 * (1 - p_disease)
    
    posterior = numerator / denominator
    
    print(f"--- 計算結果 ---")
    print(f"事前確率 p: {p_disease}")
    print(f"尤度 L(1) [sの積]: {L1:.5f}")
    print(f"尤度 L(0) [fの積]: {L0:.5f}")
    print(f"比 m = (L0*(1-p)) / (L1*p): {(L0*(1-p)) / (L1*p):.5f}")
    print(f"事後確率 P(D=1|E): {posterior:.5f}")
    
    return posterior

# 例：p=0.001, s1=0.999, f1=0.001 の場合（1回検査）
print("【問1：1回陽性】")
solve_medical_test(0.001, 0.999, 0.001)

# 例：上記に加え、2回目（1回目陽性後）の s2=0.95, f2=0.05 の場合
print("\n【問2：2回連続陽性】")
solve_medical_test(0.001, 0.999, 0.001, 0.95, 0.05)